In [ ]:
#!pip install hanlp
#!pip install simalign

In [439]:
import os, csv
# public
import hanlp
import pandas as pd
from tqdm import tqdm
from simalign import SentenceAligner
# from google.cloud import translate_v2 as translate

In [612]:
src_lan = 'en'
# target language
tgt_lan = 'ES'
language = "Spanish"

RESOURCE = 'res'
RESULTS = os.path.join(RESOURCE, 'results')
TRANS_FILE = os.path.join(RESULTS, 'sentences-'+language+'-Translations.tsv')
TOKENS_FILE = os.path.join(RESULTS, 'tokens-'+language+'.tsv')
ALIGN_SENT_FILE = os.path.join(RESULTS, 'sentences_aligned-'+language+'.tsv')
ALIGN_TOKEN_FILE = os.path.join(RESULTS, 'tokens_aligned-'+language+'.tsv')

DATA = os.path.join(RESOURCE, 'data')
KEY = os.path.join(RESOURCE, 'key')
RAW_SENSE_TSV = os.path.join(DATA, 'process_s15.tsv')
RAW_TOKENS_TSV =  os.path.join(DATA, 'tokens-English.tsv')
RAW_SENTENCE_TSV = os.path.join(DATA, 'sentences-English.tsv')
JSON_GOOGLE_APPLICATION_CREDENTIALS = os.path.join(KEY, 'tonal-works-420505-eda807c7cc52.json')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = JSON_GOOGLE_APPLICATION_CREDENTIALS


In [652]:

src_lan = 'en'
# target language
tgt_lan = 'ES'
language = "Spanish"

RESOURCE = '../..'
RESULTS =  RESOURCE+'/data/'+language
TRANS_FILE = os.path.join(RESULTS, 'gold-sentences-'+language+'.tsv')
TOKENS_FILE = os.path.join(RESULTS, 'tokens_gold_'+language+'.tsv')
ALIGN_SENT_FILE = os.path.join(RESULTS, 'sentences_aligned-'+language+'.tsv')
ALIGN_TOKEN_FILE = os.path.join(RESULTS, 'tokens_aligned-'+language+'.tsv')

DATA = os.path.join(RESOURCE, 'data')
KEY = os.path.join(RESOURCE, 'key')
RAW_SENSE_TSV = os.path.join(DATA, 'process_s15.tsv')
RAW_TOKENS_TSV =  os.path.join(DATA, 'tokens-English.tsv')
RAW_SENTENCE_TSV = os.path.join(DATA, 'sentences-English.tsv')
JSON_GOOGLE_APPLICATION_CREDENTIALS = os.path.join(KEY, 'tonal-works-420505-eda807c7cc52.json')
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = JSON_GOOGLE_APPLICATION_CREDENTIALS

# DATA

In [653]:
# raw
raw_sense_df = pd.read_csv(RAW_SENSE_TSV, delimiter='\t', header=None, quoting=csv.QUOTE_NONE)
raw_sense_df.head()

,0,1,2,3,4
0,d001.s001.t001,This,NaN,X,NaN
1,d001.s001.t002,document,document,N,bn:00028015n
2,d001.s001.t003,is,be,V,NaN
3,d001.s001.t004,a,NaN,X,NaN
4,d001.s001.t005,summary,summary,N,bn:00075142n


In [654]:
sentence_df = pd.read_csv(RAW_SENTENCE_TSV, delimiter='\t', header=None, quoting=csv.QUOTE_NONE)
sentence_df.head()
sents = sentence_df[1].tolist()
sents

['English',
 'This document is a summary of the European Public Assessment Report ( EPAR ) .',
 'It explains how the Committee for Medicinal Products for Human Use ( CHMP ) assessed the studies performed, to reach their recommendations on how to use the medicine .',
 'If you need more information about your medical condition or your treatment, read the Package Leaflet ( also part of the EPAR ) or contact your doctor or pharmacist .',
 'If you want more information on the basis of the CHMP recommendations, read the Scientific Discussion ( also part of the EPAR ) .',
 'What is Alimta ?',
 'Alimta is a powder that is made up into a solution for infusion ( drip into a vein ) .',
 'It contains the active substance pemetrexed .',
 'What is Alimta used for ?',
 'Alimta is used to treat two types of cancer: pleural mesothelioma ( cancer of the lining of the lungs that is usually caused by exposure to asbestos ) .',
 'Alimta is used together with cisplatin ( another anticancer medicine ) when t

In [655]:
sids = sentence_df[0].tolist()
# sids = [i.replace('d', 'i') for i in sids]
# sids[:4]

In [656]:
trans_df = pd.read_csv(TRANS_FILE, delimiter='\t', quoting=csv.QUOTE_NONE)
trans_df.head()

,ID,English,Spanish
0,d001.s001,This document is a summary of the European Pub...,En el presente documento se resume el Informe ...
1,d001.s002,It explains how the Committee for Medicinal Pr...,En él se explica cómo el Comité de Medicamento...
2,d001.s003,If you need more information about your medica...,Si desea más información sobre su enfermedad o...
3,d001.s004,If you want more information on the basis of t...,Si desea más información sobre el fundamento e...
4,d001.s005,What is Alimta ?,¿Qué es Alimta ?


In [657]:
tkns_df = pd.read_csv(TOKENS_FILE, delimiter='\t', quoting=csv.QUOTE_NONE)
tkns_df.head()

,Token ID,Token,Lemma,POS
0,es001.s001.t001,En,NaN,X
1,es001.s001.t002,el,NaN,X
2,es001.s001.t003,presente,presente,J
3,es001.s001.t004,documento,documento,N
4,es001.s001.t005,se,NaN,X


In [658]:
tgt_tkns = tkns_df['Token'].tolist()
len(tgt_tkns)

2803

# ALIGNMENT

In [659]:
device = 'cpu'
layer = 8
# "bert": "bert-base-multilingual-cased",
# "xlmr": "xlm-roberta-base"
model = 'xlmr'
# mwmf (Match), inter (ArgMax), itermax (IterMax)
mode = 'itermax'
matching_methods = 'mai'
token_type='bpe'
aligner = SentenceAligner(
    model=model
    , token_type=token_type
    , matching_methods=matching_methods
    , layer=layer
    , device=device
)

/usr/local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-07-08 10:27:31,285 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: xlm-roberta-base


In [660]:
# english token sequences
idxes = raw_sense_df[0].tolist()
tokens = raw_sense_df[1].tolist()

en_sents = []
# en_sents2 = trans_df["English"].tolist()
pre_sid = None
s = []
for i, t in zip(idxes, tokens):
    sid = i.split('.')[1]
    if sid != pre_sid:
        if s:
            en_sents.append(s)
        pre_sid = sid
        s = [t]
    else:
        s.append(t)
en_sents.append(s)
len(en_sents)
# for x in range(len(en_sents)):
#     print(" ".join(en_sents[x]),en_sents2[x])
#     if " ".join(en_sents[x]) != en_sents2[x]:
#         print(x)

133

In [661]:
print(en_sents[92])

['The', 'Foundation', 'organised', ',', 'together', 'with', 'the', 'European', 'Commission', ',', 'the', 'Spanish', 'Ministry', 'of', 'Social', 'Affairs', 'and', 'the', 'Galician', 'regional', 'government', ',', 'a', 'European', 'conference', 'on', 'social', 'exclusion', ':', 'a', 'major', 'challenge', 'for', 'public', 'welfare', 'services', '.']


In [662]:
# Italian token sequences
idxes = tkns_df['Token ID'].tolist()
tokens = tkns_df['Token'].tolist()

tgt_sents = []
pre_sid = None
s = []
for i, t in zip(idxes, tokens):
    sid = i.split('.')[1]
    if sid != pre_sid:
        if s:
            tgt_sents.append(s)
        pre_sid = sid
        s = [t]
    else:
        s.append(t)
tgt_sents.append(s)
len(tgt_sents)

134

In [663]:
print(tgt_sents[92])

['La', 'Fundación', 'organizó', ',', 'junto con', 'la', 'Comisión', 'Europea', ',', 'el', 'Ministerio', 'español', 'de', 'Asuntos', 'Sociales', 'y', 'la', 'Comunidad', 'Autónoma', 'de', 'Galicia', ',', 'una', 'conferencia', 'europea', 'sobre', 'el', 'tema', '""""', 'Exclusión', 'social', ':', 'Un', 'gran', 'reto', 'para', 'los', 'servicios', 'públicos', 'de', 'protección', 'social', '""""', '.']


In [664]:
# do the alignment
aligns = []
for en_s, tgt_s in zip(tqdm(en_sents), tgt_sents):
    # print(en_s,tgt_s,'\n')
    alignments = aligner.get_word_aligns(en_s, tgt_s)
    align = ''
    for w1, w2 in alignments[mode]:
        align += f'{w1}-{w2} '
    align = align.strip()
    aligns.append(align)

100%|██████████| 133/133 [00:50<00:00,  2.63it/s]


In [665]:
aligns[0]

'0-2 1-3 2-4 3-1 4-5 5-10 6-6 7-9 8-8 9-11 10-7 11-12 12-13 13-14 14-15'

In [668]:
# output
print(len(aligns))
trans_df['English-'+language+' Alignment'] = aligns
trans_df.head()

133


,ID,English,Spanish,English-Spanish Alignment
0,d001.s001,This document is a summary of the European Pub...,En el presente documento se resume el Informe ...,0-2 1-3 2-4 3-1 4-5 5-10 6-6 7-9 8-8 9-11 10-7...
1,d001.s002,It explains how the Committee for Medicinal Pr...,En él se explica cómo el Comité de Medicamento...,0-1 1-2 1-3 2-4 3-5 4-6 5-7 6-8 6-11 7-8 8-9 9...
2,d001.s003,If you need more information about your medica...,Si desea más información sobre su enfermedad o...,0-0 1-15 2-1 3-2 4-3 5-4 6-5 8-6 9-7 11-9 12-1...
3,d001.s004,If you want more information on the basis of t...,Si desea más información sobre el fundamento e...,0-0 1-19 2-1 3-2 4-3 5-4 6-5 7-6 9-15 10-16 11...
4,d001.s005,What is Alimta ?,¿Qué es Alimta ?,0-0 1-1 2-2 3-3


In [669]:
trans_df.to_csv(ALIGN_SENT_FILE, sep='\t', index=False)

# ALIGN TOKENS IN TOKEN FILE

In [670]:
tkns_df.head()


,Token ID,Token,Lemma,POS
0,es001.s001.t001,En,NaN,X
1,es001.s001.t002,el,NaN,X
2,es001.s001.t003,presente,presente,J
3,es001.s001.t004,documento,documento,N
4,es001.s001.t005,se,NaN,X


In [679]:
aligned_tks_df = pd.read_csv(RAW_TOKENS_TSV, delimiter='\t', quoting=csv.QUOTE_NONE)
aligned_tks_df.head()

,Token ID,Token,Lemma,POS,Sense
0,d001.s001.t001,This,NaN,X,NaN
1,d001.s001.t002,document,document,N,bn:00028015n
2,d001.s001.t003,is,be,V,NaN
3,d001.s001.t004,a,NaN,X,NaN
4,d001.s001.t005,summary,summary,N,bn:00075142n


In [680]:
aligned_tks_df['Token ID '+tgt_lan] = None
aligned_tks_df['Token '+tgt_lan] = None
aligned_tks_df['Lemma '+tgt_lan] = None
aligned_tks_df['POS '+tgt_lan] = None
aligned_tks_df['Sense '+tgt_lan] = None

In [681]:
aligned_tks_df.head()

,Token ID,Token,Lemma,POS,Sense,Token ID ES,Token ES,Lemma ES,POS ES,Sense ES
0,d001.s001.t001,This,NaN,X,NaN,None,None,None,None,None
1,d001.s001.t002,document,document,N,bn:00028015n,None,None,None,None,None
2,d001.s001.t003,is,be,V,NaN,None,None,None,None,None
3,d001.s001.t004,a,NaN,X,NaN,None,None,None,None,None
4,d001.s001.t005,summary,summary,N,bn:00075142n,None,None,None,None,None


In [682]:
trans_df.head()

,ID,English,Spanish,English-Spanish Alignment
0,d001.s001,This document is a summary of the European Pub...,En el presente documento se resume el Informe ...,0-2 1-3 2-4 3-1 4-5 5-10 6-6 7-9 8-8 9-11 10-7...
1,d001.s002,It explains how the Committee for Medicinal Pr...,En él se explica cómo el Comité de Medicamento...,0-1 1-2 1-3 2-4 3-5 4-6 5-7 6-8 6-11 7-8 8-9 9...
2,d001.s003,If you need more information about your medica...,Si desea más información sobre su enfermedad o...,0-0 1-15 2-1 3-2 4-3 5-4 6-5 8-6 9-7 11-9 12-1...
3,d001.s004,If you want more information on the basis of t...,Si desea más información sobre el fundamento e...,0-0 1-19 2-1 3-2 4-3 5-4 6-5 7-6 9-15 10-16 11...
4,d001.s005,What is Alimta ?,¿Qué es Alimta ?,0-0 1-1 2-2 3-3


In [683]:
def process_nan(s):
  if type(s)!=type(''):
    return ''
  return s

In [684]:
for i, row in trans_df.iterrows():
    print(row.iloc[0])
    sent_i = row.iloc[0]
    # print(sent_i)
    tgt_sent_i = sent_i.replace("d","es")
    # sent_i = sent_i.replace("i","d")
    for alignment in row['English-'+language+' Alignment'].split():
        w1, w2 = alignment.split('-')
        en_id = f'{sent_i}.t{"%03d" % (int(w1)+1)}'
        tgt_id = f'{tgt_sent_i}.t{"%03d" % (int(w2)+1)}'
        row_en = aligned_tks_df[aligned_tks_df['Token ID'] == en_id]
        row_tgt = tkns_df[tkns_df['Token ID'] == tgt_id]
        # print(aligned_tks_df['Token ID'] )
        # print(row_tgt)
        # print(row_en.iloc[0]['Token ID '+tgt_lan])
        if row_en.iloc[0]['Token ID '+tgt_lan] is not None:
            aligned_tks_df.loc[row_en.index, 'Token ID '+tgt_lan] += '➕' + row_tgt.iloc[0]['Token ID']
            aligned_tks_df.loc[row_en.index, 'Token '+tgt_lan] += '➕' + row_tgt.iloc[0]['Token']
            if aligned_tks_df.loc[row_en.index, 'POS '+tgt_lan].item() != process_nan(row_tgt.iloc[0]['POS']):
                aligned_tks_df.loc[row_en.index, 'POS '+tgt_lan] += '➕' + process_nan(row_tgt.iloc[0]['POS'])
            if aligned_tks_df.loc[row_en.index, 'Sense '+tgt_lan].item() != process_nan(row_en.iloc[0]['Sense']):
                aligned_tks_df.loc[row_en.index, 'Sense '+tgt_lan] += '➕' + process_nan(row_en.iloc[0]['Sense'])
        else:
            # print(row_en.index.values)
            aligned_tks_df.loc[row_en.index.values[0], 'Token ID '+tgt_lan] = row_tgt.iloc[0]['Token ID']
            # print(aligned_tks_df.loc[row_en.index, 'Token ID '+tgt_lan],row_tgt.iloc[0]['Token ID'],'\n\n')
    
            aligned_tks_df.loc[row_en.index.values[0], 'Token '+tgt_lan] = row_tgt.iloc[0]['Token']
            aligned_tks_df.loc[row_en.index.values[0], 'Lemma '+tgt_lan] = row_tgt.iloc[0]['Lemma']
            aligned_tks_df.loc[row_en.index.values[0], 'POS '+tgt_lan] = row_tgt.iloc[0]['POS']
            aligned_tks_df.loc[row_en.index.values[0], 'Sense '+tgt_lan] = row_en.iloc[0]['Sense']

d001.s001
d001.s002
d001.s003
d001.s004
d001.s005
d001.s006
d001.s007
d001.s008
d001.s009
d001.s010
d001.s011
d001.s012
d001.s013
d001.s014
d001.s015
d001.s016
d001.s017
d001.s018
d001.s019
d001.s020
d001.s021
d001.s022
d001.s023
d001.s024
d001.s025
d001.s026
d001.s027
d001.s028
d001.s029
d001.s030
d001.s031
d001.s032
d001.s033
d001.s034
d001.s035
d001.s036
d001.s037
d002.s001
d002.s002
d002.s003
d002.s004
d002.s005
d002.s006
d002.s007
d002.s008
d002.s009
d002.s010
d002.s011
d002.s012
d002.s013
d002.s014
d002.s015
d002.s016
d002.s017
d002.s018
d002.s019
d002.s020
d002.s021
d002.s022
d002.s023
d002.s024
d002.s025
d002.s026
d002.s027
d002.s028
d002.s029
d002.s030
d002.s031
d002.s032
d002.s033
d002.s034
d002.s035
d002.s036
d002.s037
d002.s038
d002.s039
d002.s040
d002.s041
d002.s042
d002.s043
d002.s044
d002.s045
d002.s046
d002.s047
d002.s048
d002.s049
d002.s050
d002.s051
d002.s052
d003.s002
d003.s003
d003.s004
d003.s005
d003.s006
d003.s007
d003.s008
d003.s009
d003.s010
d003.s011
d003.s012


IndexError: single positional indexer is out-of-bounds

In [677]:
# aligned_tks_df[]
aligned_tks_df.head(20)

,Token ID,Token,Lemma,POS,Sense,Token ID ES,Token ES,Lemma ES,POS ES,Sense ES
0,d001.s001.t001,This,NaN,X,NaN,es001.s001.t003,presente,presente,J,NaN
1,d001.s001.t002,document,document,N,bn:00028015n,es001.s001.t004,documento,documento,N,bn:00028015n
2,d001.s001.t003,is,be,V,NaN,es001.s001.t005,se,NaN,X,NaN
3,d001.s001.t004,a,NaN,X,NaN,es001.s001.t002,el,NaN,X,NaN
4,d001.s001.t005,summary,summary,N,bn:00075142n,es001.s001.t006,resume,resumir,V,bn:00075142n
5,d001.s001.t006,of,NaN,X,NaN,es001.s001.t011,de,NaN,X,NaN
6,d001.s001.t007,the,NaN,X,NaN,es001.s001.t007,el,NaN,X,NaN
7,d001.s001.t008,European,european,J,bn:00102440a,es001.s001.t010,Europeo,europeo,J,bn:00102440a
8,d001.s001.t009,Public,public,J,bn:00109211a,es001.s001.t009,Público,público,J,bn:00109211a
9,d001.s001.t010,Assessment,assessment,N,bn:00006502n,es001.s001.t012,Evaluación,evaluación,N,bn:00006502n


In [678]:
aligned_tks_df.to_csv(ALIGN_TOKEN_FILE, sep='\t', index=False)